In [3]:
import pandas as pd

In [12]:
no_of_contraints = 3
no_of_var = 4
max_or_min = "Max"
A = [  
    "-1 1 1 3 <= 1",
    "1 -1 2 1 <= 1",
]

for constraint in A:
    split_list = constraint.split(" ")
    coef = [int(x) for x in split_list[0:no_of_var]]

    eq = split_list[no_of_var]

    val = split_list[-1]
    
    print(coef, eq, val)

[-1, 1, 1, 3] <= 1
[1, -1, 2, 1] <= 1


In [2]:
# REQUIRED INPUTS
no_of_var_incl_slack = 6



In [78]:
# ASSUMPTION: ALREADY CONVERTED TO CANONICAL FORM 
# ASSUMPTION: MAX QUESTION 



class Simplex():
    
    """Creates a LP problem class that can be solved using the simplex method. 
        @param: number of variables and number of constraints"""
    def __init__(self, no_of_var, no_of_constaints):
        self.no_of_var = no_of_var
        # Create columns with variable numbers (e.g. x1, x2)
        self.variables = ["x"+str(i) for i in range(1,no_of_var+no_of_constaints+1)]
        self.variables.append('b')
        # Create simplex table 
        self.table = pd.DataFrame(columns = self.variables)
        
        # tracks basic variable numbers 
        self.no_of_constraints = no_of_var
        # Stores whether objective function has been entered or not
        self.obj_func_given = False
    
    """Returns the table of the LP problem"""
    def print_table(self):
        print(self.table)
        return

    
    """Adds a constraint to the LP problem """
    def add_constraint(self, x):
        self.no_of_constraints +=1
        assert(len(x)==len(self.variables))
        constraint = pd.DataFrame(data=[x], columns= self.variables, index = ["x"+str(self.no_of_constraints)])
        if (self.obj_func_given==True):
            obj_func = self.table.iloc[-1]
            self.table = self.table.drop(['obj_func'])
            self.table = self.table.append(constraint)
            self.table = self.table.append(obj_func)
            
        else:
            self.table = self.table.append(constraint)
    
    """Adds the objective function to the LP problem"""
    def add_obj_function(self, x):
        assert(self.obj_func_given == False)
        self.obj_func_given = True
        assert(len(x)==len(self.variables)-1)
        data = x + [0]
        data = [[-i for i in data]]
        obj_func = pd.DataFrame(data = data, columns = self.variables, index = ['obj_func'])
        self.table = self.table.append(obj_func)
    
    """Returns the maximum reduced cost and the corresponding variable"""
    def get_max_reduced_cost(self):
        reduced_cost_list = self.table.iloc[-1]
        max_reduced_cost = 0
        max_reduced_cost_var = None
        for var in self.table.columns[0:-1]:
            rc = reduced_cost_list.loc[var]
            if rc < max_reduced_cost:
                max_reduced_cost = rc
                max_reduced_cost_var = var
                
        return max_reduced_cost, max_reduced_cost_var
    
    """Return the optimized solution for LP problem"""
    def get_solution(self):
        
        # extract greatest reduced cost and corresponding non-basic variable
        max_reduced_cost, max_reduced_cost_var = self.get_max_reduced_cost()       
        
        # Loops everytime we can continue to optimise the problem 
        # AKA objective function can still be maximised 
        
        while (max_reduced_cost < 0):

            # y vector 
            y = self.table[max_reduced_cost_var].iloc[0:-1]
            # b vector 
            b = self.table['b'].iloc[0:-1]
            
            
            #COMPLETE THE RATIO TEST
            min_ratio = None
            swap_var = None
            # for every basic variable 
            for x in self.table.index[0:-1]:
                # will have a ratio if y value is not 0
                if (y[x]!=0):
                    curr_ratio = b[x]/y[x]
                # if y value is 0, basic var will not change regardless of how much non-basic var changes 
                else:
                    continue
                # y value pos means basic var will decrease 
                # store ratio of current basic var if it's the smallest 
                if (y[x]>0 and (min_ratio==None or curr_ratio < min_ratio)):
                    min_ratio = curr_ratio
                    swap_var = x
            
            # No basic var can become infeasible 
            # Therefore, problem can be maximised forever 
            if (swap_var == None):
                print("Unbounded objective function")
                return
        
            # SWAP OUT BASIC VAR, SWAP IN NON-BASIC VAR  
            # stores the value which will become the leading 1
            pivot = self.table.loc[swap_var][max_reduced_cost_var]
            
            # changes the index of table so that non-basic var is now a basic-var 
            new_index_list = self.table.index.tolist()
            index_val = new_index_list.index(swap_var)
            new_index_list[index_val] = max_reduced_cost_var
            self.table.index = new_index_list
            
            # make row have the desired leading 1 
            self.table.loc[max_reduced_cost_var] = self.table.loc[max_reduced_cost_var]/pivot
            
            # perform row operations 
            for index, row in self.table.iterrows():
                if index == max_reduced_cost_var:
                    continue
                factor = self.table.loc[index, max_reduced_cost_var]
                self.table.loc[index] = self.table.loc[index] - factor*self.table.loc[max_reduced_cost_var]
                
            print(self.table)
            
            # extract greatest reduced cost and corresponding non-basic variable
            max_reduced_cost, max_reduced_cost_var = self.get_max_reduced_cost()
            print("non-basic var being swapped in:", max_reduced_cost_var)
            
        
        self.max_obj_func = self.table.loc['obj_func']['b']
        
        solution_array = []
        for var in self.table.columns[0:-1]:
            all_zeroes = True
            b_value = None
            for value in self.table.iloc[:-1][var]:
                if value == 1 and all_zeroes:
                    all_zeroes = False
                    b_value = self.table.loc[var]['b']
                elif value == 0:
                    continue
                else:
                    b_value = 0
                    break
            solution_array.append(b_value)
            
        self.solution = pd.Series(solution_array, index=self.table.columns[0:-1])
                    
        return
        
        
problem1 = Simplex(2, 4)
problem1.add_constraint([-1,1,1,0,0,0,2])
problem1.add_constraint([1,-1,0,1,0,0,4])
problem1.add_constraint([1,2,0,0,1,0,10])
problem1.add_constraint([3,1,0,0,0,1,15])
problem1.add_obj_function([1,1,0,0,0,0])
problem1.print_table()
print()
problem1.get_solution()

          x1  x2 x3 x4 x5 x6   b
x3        -1   1  1  0  0  0   2
x4         1  -1  0  1  0  0   4
x5         1   2  0  0  1  0  10
x6         3   1  0  0  0  1  15
obj_func  -1  -1  0  0  0  0   0

         x1 x2 x3 x4 x5 x6  b
x3        0  0  1  1  0  0  6
x1        1 -1  0  1  0  0  4
x5        0  3  0 -1  1  0  6
x6        0  4  0 -3  0  1  3
obj_func  0 -2  0  1  0  0  4
non-basic var being swapped in: x2
         x1 x2 x3    x4 x5    x6     b
x3        0  0  1     1  0     0     6
x1        1  0  0  0.25  0  0.25  4.75
x5        0  0  0  1.25  1 -0.75  3.75
x2        0  1  0 -0.75  0  0.25  0.75
obj_func  0  0  0  -0.5  0   0.5   5.5
non-basic var being swapped in: x4
         x1 x2 x3 x4   x5   x6  b
x3        0  0  1  0 -0.8  0.6  3
x1        1  0  0  0 -0.2  0.4  4
x4        0  0  0  1  0.8 -0.6  3
x2        0  1  0  0  0.6 -0.2  3
obj_func  0  0  0  0  0.4  0.2  7
non-basic var being swapped in: None
